# Example Code for Estimating the Ground State Energy of Hydroxyl (·OH)

## Basic Installation

Install required package, we highly recommend participant to use qiskit platform, or at least participants can finish preprocessing at other platform and transfer the circuit to qiskit format, since our noise model is from IBM real machine backend and we restricted some algorithmic seeds which could be varied from different platform.

In [1]:
!pip install qiskit
!pip install qiskit-nature[pyscf] -U

In [2]:
!pip install qiskit_aer

In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
import numpy as np
import pylab
import qiskit.providers
from qiskit import Aer,pulse, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
import time

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as seeds that requires to run, and the result will be calculated as the average of results from each seed. And please use shots as 4000.

In [4]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
iterations = 125
shot = 4000

## Generate Hamiltonian and Pauli String

At this step, the example code uses PySCF to generate the hamiltonian of hydroxyl with basis function as 'sto3g' to fit the spin orbital, then uses JordanWignerMapper to map the fermionic terms to pauli strings. To be noticed, other chemistry tool also allowed to be used at this step, but keep in mind to use 'sto-3g' and Jordan Wigner Mapper which should gives 12 qubits and 631 paulil terms.

In [5]:
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""

driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()

/common/home/zl606/.conda/envs/qiskit43/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [6]:
hamiltonian = qmolecule.hamiltonian
coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)
second_q_op = hamiltonian.second_q_op()

Polynomial Tensor
 "+-":
[[-3.21461222e+01  5.59899100e-01  1.87617178e-01 -4.56343387e-17
   2.41250904e-16 -1.94702445e-01]
 [ 5.59899100e-01 -7.35898345e+00 -2.46352634e-01  2.39982429e-16
  -7.57528634e-16  9.51226718e-01]
 [ 1.87617178e-01 -2.46352634e-01 -6.56995119e+00  5.63137781e-16
   7.91837340e-16 -1.09726793e+00]
 [-4.54848900e-17  2.05430655e-16  5.24773314e-16 -6.94886145e+00
   1.56026214e-15  4.12531363e-16]
 [ 2.45643545e-16 -8.91028759e-16 -4.57755709e-18  1.55483212e-15
  -6.94886145e+00 -2.07120237e-15]
 [-1.94702445e-01  9.51226718e-01 -1.09726793e+00  3.50125199e-16
  -2.87761855e-15 -4.64967973e+00]]
 "++--":
[[[[ 4.74977044e+00 -4.38465691e-01 -1.51436760e-01  4.89323067e-17
    -1.83704413e-16  1.59790984e-01]
   [-4.38465691e-01  6.47204045e-02  1.84429506e-02 -6.55656318e-18
     2.71307968e-17 -2.66865302e-02]
   [-1.51436760e-01  1.84429506e-02  2.46189939e-02 -1.60762618e-18
    -1.65835535e-18  6.40512562e-03]
   [ 4.89487958e-17 -6.44131745e-18 -2.64663

In [7]:
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
qubit_op = converter.convert(second_q_op)

/tmp/ipykernel_528114/561404713.py:2: DeprecationWarning: The QubitConverter class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . Instead you should directly use the QubitMapper instance which you used to pass into the QubitConverter as the first argument. Refer to the documentation of the qiskit_nature.second_q.mappers module for more information.
  converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)


/common/home/zl606/.conda/envs/qiskit43/lib/python3.11/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


We recommend to use classical minimum eigensolver to obtain a reference energy at this step. In case some of the classical minimum eigensolver donot directly gives nuclear repulsion energy, we give reference energies below: *Comupted Energy*: -78.75252123, *Nuclear Repulsion_energy*: 4.36537496654537. *Obtained Reference Ground State Energy*: -74.38714627.

In [24]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [25]:
result = solver.solve(qmolecule)
print(result.computed_energies)

[-78.75252123]


In [21]:
print(result.nuclear_repulsion_energy)

4.36537496654537


In [11]:
ref_value = result.computed_energies + result.nuclear_repulsion_energy
print(ref_value)

[-74.38714627]


## Construct Ansatz

At this stage, you can implement various techniques to search good ansatz architecture which is important for variational quantum algorihms. Moreover, how to obtain a good initial state is a good topic to do research, we require participant to self-reflection there techniques (include the techniques for preprocessing ansatz or initial states) with maximum 10 points, and submit a short description for used techniques, we will have three graders to evaluate the techniques.

In [12]:
ansatz = UCCSD(
    qmolecule.num_spatial_orbitals,
    qmolecule.num_particles,
    mapper,
    initial_state=HartreeFock(
        qmolecule.num_spatial_orbitals,
        qmolecule.num_particles,
        mapper,
    ),
)


Load the noise models.

In [13]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.providers.fake_provider import *
import pickle
from qiskit_aer.primitives import Sampler


In [14]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model_kolkata = pickle.load(file)
with open('NoiseModel/fakecairo.pkl', 'rb') as file:
    noise_model_cairo = pickle.load(file)
with open('NoiseModel/fakemontreal.pkl', 'rb') as file:
    noise_model_montreal = pickle.load(file)
noise_model1 = noise.NoiseModel()
noise_model2= noise.NoiseModel()
noise_model3 = noise.NoiseModel()
noise_model_kolkata = noise_model1.from_dict(noise_model_kolkata)
noise_model_cairo = noise_model2.from_dict(noise_model_cairo)
noise_model_montreal = noise_model3.from_dict(noise_model_montreal)

In [15]:
sampler = Sampler(
    backend_options = {
        'method': 'statevector',
        'device': 'CPU',
        'noise_model': noise_model_kolkata
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler':seed_transpiler
    }
)

### Group measurement

In [16]:
from utils.varsaw import parseHamiltonian, group_measurements, varsaw_expectation

In [17]:
import pickle
# run once!
h, first_term = parseHamiltonian('Hamiltonian/OHhamiltonian.txt')
# measurements, measurement_dict = group_measurements(h)
# filehandler = open(b"142observables.obj","wb")
# pickle.dump((measurements, measurement_dict),filehandler)

filehandler = open(b"142observables.obj","rb")
measurements, measurement_dict = pickle.load(filehandler)


In [20]:
test_ansatz = QuantumCircuit(12, 12)
test_ansatz.x(0)
test_ansatz.x(1)
test_ansatz.x(2)
test_ansatz.x(3)
test_ansatz.x(6)
test_ansatz.x(7)
test_ansatz.x(8)
test_ansatz.x(9)
computed_energies = varsaw_expectation(test_ansatz, measurements, measurement_dict, first_term, h, sampler)

In [29]:
system_model = FakeMontreal()
transpiled_circuit = transpile(test_ansatz, backend=system_model)
print(test_ansatz.qasm())
print(transpiled_circuit.qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[12];
creg c[12];
x q[0];
x q[1];
x q[2];
x q[3];
x q[6];
x q[7];
x q[8];
x q[9];

OPENQASM 2.0;
include "qelib1.inc";
qreg q[27];
creg c[12];
x q[0];
x q[1];
x q[2];
x q[3];
x q[6];
x q[7];
x q[8];
x q[9];



## Calculate the Accuracy (Most Important Metric)

In [26]:
estimated = computed_energies + result.nuclear_repulsion_energy
error_rate = abs(abs(ref_value - estimated) / ref_value * 100)
print("Error rate: %f%%" % (error_rate))
print("Escore: %f" % (100-error_rate))

Error rate: 1.959242%
Escore: 98.040758


In [20]:
from arch import load_coupling_map
from time import time as ctime
from utils.parallel_bl import gate_count_oriented_scheduling
from utils.synthesis_broccoli import synthesis
from qiskit import QuantumCircuit, transpile

def Tetris_Montreal(parr, use_bridge=False):
    print('Tetris passes, Our schedule, Our synthesis, montreal', flush=True)
    lnq = len(parr[0][0]) # lnq: number of qubits
    length = lnq // 2 # `length' is a hyperparameter, and can be adjusted for best performance. Here we keep `length' fixed for simplicity.
    coup = load_coupling_map('montreal')
    t0 = ctime()
    a2 = gate_count_oriented_scheduling(parr) # parr is the pauli_block list
    # a2 = [[block] for block in parr]
    qc, metrics = synthesis(a2, arch='montreal', use_bridge=use_bridge)
    pnq = qc.num_qubits
    print(pnq)
    print('Tetris, Time costed:', ctime()-t0, flush=True)
    t0 = ctime()
    qc2 = transpile(qc, basis_gates=['u3', 'cx'], coupling_map=coup, initial_layout=list(range(pnq)), optimization_level=3)
    print('Qiskit L3, Time costed:', ctime()-t0, flush=True)
    return qc2


In [21]:
import ast
from benchmark.mypauli import pauliString

pauli_blocks = []
for pauli_list in ansatz._operators: # load all pauli blocks to a list
    block = ast.literal_eval(pauli_list._primitive._pauli_list.__str__())
    block = [pauliString(ps) for ps in block]
    pauli_blocks.append(block)

new_ansatz = Tetris_Montreal(pauli_blocks, use_bridge=False)

Tetris passes, Our schedule, Our synthesis, montreal
27
27
Tetris, Time costed: 0.3336620330810547
Qiskit L3, Time costed: 17.822885751724243


In [27]:
with open('pauli_string.txt', 'w') as f:
    for block in pauli_blocks:
        f.write(str(block) + '\n')

In [40]:
cnt = 0
for block in pauli_blocks:
    for ps in block:
        cnt = cnt + 1
print(cnt)

640


## Obtain the Duration of Quantum Circuit

In [32]:
from qiskit.providers.fake_provider import *
backend = FakeMontreal()

In [37]:
with pulse.build(backend) as my_program1:
  pulse.call(ansatz)

In [38]:
my_program1.duration 

19696160